In [31]:
import pickle
import numpy as np

with open('train_qa.txt','rb') as f:
    train_data=pickle.load(f)

In [3]:
with open('test_qa.txt','rb') as f:
    test_data=pickle.load(f)

In [4]:
type(test_data)

list

In [5]:
all_data=test_data+ train_data

In [6]:
len(all_data)

11000

In [7]:
vocab =set()
for story,question,answer in all_data:
    vocab=vocab.union(set(story))
    vocab=vocab.union(set(question))

In [8]:
vocab.add('no')
vocab.add('yes')

In [9]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [11]:
vocab_len=len(vocab)+1

In [12]:
vocab_len

38

In [13]:
#Longest story

all_story_lens=[len(data[0]) for data in all_data]
max_story_len=max(all_story_lens)

In [14]:
#Longest question
all_question_lens=[len(data[1]) for data in all_data]
max_question_len=max(all_question_lens)

In [18]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [19]:
tokenizer=Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [20]:
tokenizer.word_index

{'took': 1,
 '?': 2,
 'moved': 3,
 'sandra': 4,
 'put': 5,
 'in': 6,
 'hallway': 7,
 'the': 8,
 'up': 9,
 'down': 10,
 'office': 11,
 'there': 12,
 'grabbed': 13,
 'got': 14,
 'left': 15,
 'football': 16,
 'garden': 17,
 'no': 18,
 'apple': 19,
 'went': 20,
 'milk': 21,
 'yes': 22,
 '.': 23,
 'to': 24,
 'mary': 25,
 'journeyed': 26,
 'discarded': 27,
 'kitchen': 28,
 'bedroom': 29,
 'picked': 30,
 'travelled': 31,
 'back': 32,
 'bathroom': 33,
 'dropped': 34,
 'john': 35,
 'is': 36,
 'daniel': 37}

In [21]:
train_story_text=[]
train_question_text=[]
train_answers=[]

In [23]:
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [25]:
#train_story_text

In [26]:
train_story_seq =tokenizer.texts_to_sequences(train_story_text)

In [28]:
#train_story_seq

In [34]:
def vectorize_stories(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
    #Stories=X
    X=[]
    #Questions=Xq
    Xq=[]
    #Y Correct answer(yes/no)
    Y=[]
    
    for story,query,answer in data:
        x=[word_index[word.lower()] for word  in story]
        xq=[word_index[word.lower()] for word in query]
        y=np.zeros(len(word_index)+1)
        y[word_index[answer]]=1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    return (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xq,maxlen=max_question_len),np.array(Y))

In [35]:
inputs_train , queries_train , answers_train =vectorize_stories(train_data)

In [36]:
inputs_test , queries_test , answers_test =vectorize_stories(test_data)